# Chapter 10. RDDs

## Creating RDDs

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
  .appName('RDDs')\
  .getOrCreate()

24/08/15 09:00:04 WARN Utils: Your hostname, Khanhs-MAC.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
24/08/15 09:00:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 09:00:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.range(10).rdd

MapPartitionsRDD[5] at javaToPython at NativeMethodAccessorImpl.java:0

In [3]:
spark.range(10).toDF("id").rdd.map(lambda x: row[0])

PythonRDD[12] at RDD at PythonRDD.scala:53

In [4]:
spark.range(10).rdd.toDF()

DataFrame[id: bigint]

In [5]:
myCollection  = "Spark The Definitive Guide: Big Data Processing Made Simple"\
  .split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)

In [6]:
words.setName("myWords")
print(words.name())

myWords


In [7]:
words.distinct().count()

9

In [8]:
def startsWithS(individual):
    return individual.startswith("S")

In [9]:
words.filter(lambda word: startsWithS(word)).collect()

['Spark', 'Simple']

In [10]:
words2 = words.map(lambda word: (word, word[0], word.startswith("S")))

In [11]:
words.flatMap(lambda word: list(word)).take(10)

['S', 'p', 'a', 'r', 'k', 'T', 'h', 'e', 'D', 'e']

In [12]:
words.sortBy(lambda word: len(word) * -1).take(2)

['Definitive', 'Processing']

In [13]:
fiftyFiftySplit = words.randomSplit([0.5, 0.5])

In [14]:
spark.sparkContext.parallelize(range(1, 21)).reduce(lambda x, y: x + y) # 210

210

In [15]:
def wordLengthReducer(leftWord, rightWord):
  if len(leftWord) > len(rightWord):
    return leftWord
  else: 
    return rightWord

words.reduce(wordLengthReducer)

'Processing'

In [16]:
words.count()

9

In [17]:
words.countApproxDistinct(0.05)

9

In [18]:
words.countByValue()

defaultdict(int,
            {'Spark': 1,
             'The': 1,
             'Definitive': 1,
             'Guide:': 1,
             'Big': 1,
             'Data': 1,
             'Processing': 1,
             'Made': 1,
             'Simple': 1})

In [19]:
words.first()

'Spark'

In [20]:
words.cache()


myWords ParallelCollectionRDD[24] at readRDDFromFile at PythonRDD.scala:289

In [21]:

  words.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [25]:
# spark.sparkContext.setCheckpointDir("../data/flight-data/csv/2010-summary.csv")
# words.checkpoint()

In [22]:
words.pipe("wc -l").collect()

['       4', '       5']

In [26]:
words.mapPartitions(lambda part: [1]).sum() # 2

2

In [27]:
def indexedFunc(partitionIndex, withinPartIterator):
    return ["partition: {} => {}".format(partitionIndex,
      x) for x in withinPartIterator]
words.mapPartitionsWithIndex(indexedFunc).collect()

['partition: 0 => Spark',
 'partition: 0 => The',
 'partition: 0 => Definitive',
 'partition: 0 => Guide:',
 'partition: 1 => Big',
 'partition: 1 => Data',
 'partition: 1 => Processing',
 'partition: 1 => Made',
 'partition: 1 => Simple']

In [30]:
spark.sparkContext.parallelize(["Hello", "World"], 2).glom().collect()

[['Hello'], ['World']]